In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr
torch.manual_seed(42)
generator = torch.Generator()
generator.manual_seed(42)
torch.backends.cudnn.deterministic = True

from functools import partial

from src.trainer import Trainer
from src.Focal_loss import FocalLoss
from src.Model_class import Model_class
from src.Loss_class import Loss_class
from src.DataLoader import TableDatasetDF

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
BATCH_SIZE = 1024
INIT_PARAM = 264
NUM_FEATURES = 60

In [3]:
df = pd.read_csv('../data/data_fe.csv')
feature_importance = pd.read_csv('../data/feature_imporstance_WS.csv')

In [4]:
features = list(feature_importance.iloc[:NUM_FEATURES]['feature_names'])
df = df[features + ['Machine failure']]

df.to_csv('df_to_enc.csv')

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Machine failure']),
                                                 df['Machine failure'],
                                                 shuffle=True,
                                                 stratify=df['Machine failure'], random_state=42)

In [5]:
X_train = pd.DataFrame(X_train).reset_index(drop = True)
X_test = pd.DataFrame(X_test).reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [6]:
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis = 1)

train_dataset = TableDatasetDF(df_train)
val_dataset = TableDatasetDF(df_test)

In [7]:
train_dl = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

In [8]:
model = nn.Sequential(
    nn.Linear(NUM_FEATURES, INIT_PARAM),
    nn.BatchNorm1d(INIT_PARAM),
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(INIT_PARAM, int(INIT_PARAM/2)),
    nn.BatchNorm1d(int(INIT_PARAM/2)),
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(int(INIT_PARAM/2), int(INIT_PARAM/4)),
    nn.BatchNorm1d(int(INIT_PARAM/4)),
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(int(INIT_PARAM/4), int(INIT_PARAM/8)),
    nn.BatchNorm1d(int(INIT_PARAM/8)),
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(int(INIT_PARAM/8), 2)
)

In [9]:
loss = Loss_class(FocalLoss(gamma=2))
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=model,
                    device=device)

optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.95)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=40)

wandb_init_params = dict(
    name=f'fe_{INIT_PARAM}_{BATCH_SIZE}_{NUM_FEATURES}',
    project="Internship_project",
    dir = '../logs/'
)

In [10]:
trainer = Trainer(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/'
                  )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmitrii_fomin (dmitrii_fomin_uga). Use `wandb login --relogin` to force relogin


In [11]:
trainer.train_model(learning_params)
wandb.finish()

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00,  4.03it/s]


Epoch: 1 of 40, 0.053 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00,  4.97it/s]


Epoch: 2 of 40, 0.050 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00,  5.50it/s]


...

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00,  4.58it/s]


Epoch: 5 of 40, 0.043 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00,  4.39it/s]


Epoch: 6 of 40, 0.050 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00,  4.89it/s]
